In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
# Install only required packages
!pip install langchain langchain-community langchain-groq wikipedia groq -q

# Imports
import os
import wikipedia
from kaggle_secrets import UserSecretsClient
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain

# API key setup
user_secrets = UserSecretsClient()
os.environ["GROQ_API_KEY"] = user_secrets.get_secret("GROQ_API_KEY")

# LangChain LLM setup
llm = ChatGroq(model_name="llama3-70b-8192")

# Define how the LLM should summarize Wikipedia content
prompt = PromptTemplate(
    input_variables=["context"],
    template="""
Given the following Wikipedia article content, extract:
- Founder
- Year Founded
- Number of Branches/Campuses
- Number of Employees
- A 4-line summary

Content:
{context}
"""
)

# Combine Wikipedia document with the prompt
chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

# Institution Wikipedia lookup
institution_name = "T John Institute of Technology"
try:
    wiki_content = wikipedia.page(institution_name).content
    doc = Document(page_content=wiki_content)
    result = chain.invoke({"context": [doc]})
    print(result.strip() if isinstance(result, str) else result)
except wikipedia.exceptions.PageError:
    print(f"No page found for '{institution_name}'.")
except wikipedia.exceptions.DisambiguationError as e:
    print(f"Ambiguous institution name. Suggestions: {e.options[:5]}")

Here are the extracted information and summary:

**Founder:** Thomas P John
**Year Founded:** 2006
**Number of Branches/Campuses:** 1 (part of T John Group of Institutions, which has expanded to include new departments and blocks and spread over an area of 20 acres)
**Number of Employees:** Not mentioned

**4-line Summary:**
T. John Institute of Technology is a private engineering college located in Bangalore, Karnataka, India. It offers undergraduate and postgraduate programs and is affiliated to Visvesvaraya Technological University and approved by AICTE. The college has expanded to offer B.E. degrees in six streams of engineering and two postgraduate courses. It has various departments, facilities, and extracurricular activities, including a digital library, auditorium, laboratory, and hostel.
